# Computational Phenotyping

_by Thomas Walter, Mines ParisTech, Institut Curie, INSERM U900._

In this notebook, we will apply unsupervised and supervised learning techniques in order to analyze phenotypic data. The phenotypic data comes in the form of manually annotated cell morphologies: to each cell a manual annotator has assigned one phenotypic class, such as `interphase`, `prometaphase`, ... These classes are mutually exclusive.

## Data import

First, we load the training data, as saved by [CellCognition](https://www.cellcognition-project.org). CellCognition is a tool for the analysis of High Content Screening data and computational phenotyping. In particular, it is also very useful for manual annotation of cellular data. The software saves the training set data as an `arff` file. In this file, we have all features for the annotated training samples and the class of each sample, saved in the last column.

In [ ]:
from scipy.io import arff
import pandas as pd
import numpy as np

data = arff.loadarff('./data/classifier/Classifier_2016_09_27/data/features.arff')
df = pd.DataFrame(data[0])

df.head()

We next would like to load the class definitions. They are saved as a tab-delimited text file, containing:
- the label of the class
- the string representation of the class
- the color that has been chosen during annotation

We will therefore import these definitions by reading the `csv` file.

In [ ]:
import csv

In [ ]:
fp = open('./data/classifier/Classifier_2016_09_27/class_definition.txt', 'r')
reader = csv.reader(fp, delimiter='\t')
name_to_label = {}
label_to_name = {}
lut = {}
for label, class_name, class_color in reader:
    name_to_label[class_name] = int(label)
    label_to_name[int(label)] = class_name
    lut[int(label)] = class_color

In [ ]:
label_to_name

The classes have the following meaning: `Interphase` is normally related to the cell cycle and regroups all non-mitotic phases (G1, S, G2). Here however, we take `Interphase` as a shortcut for normal interphase, meaning that interphase nuclei are nuclei of normal, non-dividing cells. `Elongated` nuclei are also non-dividing but have a an elongated shape, `Polylobed` is an abnormal nuclear morphology usually resulting from segregation defects (i.e. the nuclei are not properly distributed to the daughter nuclei during division and / or the nuclear envelope does not form correctly). `Apoptosis` is cell death, and the other classes correspond to morphologies appearing during division: `Prometaphase`, `Metaphase` and `Anaphase` are normal morphologies during division, `MAP` stands for Metaphase Alignment Problem and indicates uncomplete alignment of chromosomes in the metaphase plate. 

The `arff`-file we have imported contains both the class labels and the features. Obviously, we need to separate class labels from features in order to train classifiers, etc. We will therefore create a design matrix $X$ containing all columns of the data frame except the last and an $y$ vector containing the class labels (where we take the labels from class_definition, rather than the string representation).

In [ ]:
y = np.array([name_to_label[x.decode("utf-8").strip('\'')] for x in df['class']])

In [ ]:
X = df.drop('class', axis=1)

In [ ]:
X.shape

We have now built our training set. We have 875 samples (annotated nuclei) and 235 features for each.

## Data Cleaning and Preprocessing

The next step is to check whether there are `Nan`'s in the data. `Nan` means "not a number"; this typically happens if a feature value was not defined for some reason (e.g. division by zero). 

In [ ]:
nb_nans = np.isnan(X).sum().sum()

In [ ]:
print ('Number of Nan in the data: %i' % nb_nans)

We see that there are no `Nan`s, so we can proceed. It must be noted that some learning algorithms can deal with `Nan`s, but often it is worth thinking about how to avoid them altogether, if possible. This is of course only possible if the data scientist can control the entire workflow.

The next thing to check is whether there are features with zero-variance. They certainly contain absolutely no information (they are constant over all classes), and nothing can be learned from them. 

In [ ]:
X.columns[np.where(np.std(X)==0)]

Apparently, there is one feature that does not vary for the entire data set, we can therefore safely remove it. Again, for many Machine Learning algorithms, it does not hurt to keep the feature, but for some it can be detrimental. We therefore prefer to remove the corresponding column.

In [ ]:
X = df.drop(['dyn_distance_radius_3', 'class'], axis=1)
print(X.shape)

We next normalize the data. We clearly saw that the different features we are using have very different ranges (some are between 0 and 1, others can reach very high values). Again, data normalization is not necessary for all machine learning algorithms. For some however, it is absolutely essential. Imagine for instance that you just calculate the Euclidean distance between data points $\|x_i - x_j\|_2$ (for instance for the purpose of clustering or visualization). In this case a feature with absolute values would completely dominate this metric. Here we use a standard normalization (zero mean and unity variance). If $x_j$ is one feature (column of $X$), we define $x^{norm}_j$ as:

\begin{equation}
x^{norm}_j = \frac{x_j - \mu(x_j)}{\sigma_{x_j}}  
\end{equation}

In [ ]:
Xnorm=(X-X.mean())/X.std()

In [ ]:
np.mean(Xnorm).head()

In [ ]:
np.std(Xnorm).head()

From now on, we work only on normalized features (i.e. we omit $norm$ in the formulas and set `X` to `Xnorm`.

In [ ]:
X = Xnorm

## Visualization of the data and unsupervised analyses

We next turn to look at the data and show several ways of visually analyzing the data set. While this is often neglected in theoretical talks, looking at the data is usually an important part of a data scientist's work. It must be noted that in many real-world applications, in particular in biology, the definition of classes is not necessarily obvious and can undergo some modification during a project's lifetime. It is therefore important to get an idea about the trends in your data.

#### Plotting features

One way of looking at the data is to visualize joint distributions of features via scatterplots of feature pairs. For this, we use `matplotlib`. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def make_scatterplot(X, y, feature1, feature2, class_names):
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel(feature1)
    ax.set_ylabel(feature2)
    ax.set_title('Scatter plot: %s vs. %s' % (feature2, feature1))

    for class_name in class_names:
        class_label = name_to_label[class_name]
        class_color = lut[class_label]
        ax.scatter(X[feature1][y==class_label],
                   X[feature2][y==class_label],
                   c=class_color, 
                   s=15)
    ax.legend(class_names)
    ax.grid()

make_scatterplot(X, y, 'roisize', 'granu_open_volume_3', 
                 ['Interphase', 'Polylobed', 'Prometaphase'])

Here, we plot a texture feature (morphological granulometries of size 3) and the number of pixels, i.e. one shape and one texture descriptor. We see that these features are quite discriminative between these classes. Not surprisingly, the pixelsize is capable of distinguishing between polylobed and interphase / prometaphase and the texture feature is capable of distinguishing prometaphase from interphase / polylobed. So we see that there is signal, but we also notice that the classes are far from being separable (by taking just these two features).

Sometimes, you might be tempted to reduce the number of features by removing features with little or no discriminative power and features that are strongly correlated to other features, as they -  so the idea - convey similar information. Let us take for instance 2 features describing second order statistics (`h1_2VAR` and `h4_1VAR`) and calculate the overall correlation.

In [ ]:
np.corrcoef(X['h1_2VAR'], X['h4_2VAR'])

We thus see that we get a strong correlation of these two features (0.92) suggesting that we could safely remove one of them without loosing much information.

In [ ]:
make_scatterplot(X, y, 'h1_2VAR', 'h4_2VAR', ['Interphase', 'Prometaphase'])

<font color=red> *How do you interpret this plot? Are these features useful?*</font>

We see from this scatterplot, that in addition to being highly correlated, both features are - when taken separately - not discriminative for these two classes. However, when we consider them together, they allow us to separate the two classes with reasonable accuracy. This is therefore a good example of the danger of applying such ad hoc techniques in order to reduce features.

#### Principal Component Analysis (PCA)

Another way of visualizing the data is to find a low dimensional reprentation. There are many methods in order to achieve this goal. The two most popular methods are Principal Component Analysis (PCA) and t-Distributed Stochastic Neighbor Embedding (t-SNE). In the following, we will apply these methods on our training data.

PCA projects the feature data onto a space where all components are orthogonal to each other and have maximal variance.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('PCA (training set)')

class_names = name_to_label.keys()
for class_name in class_names:
    class_label = name_to_label[class_name]
    class_color = lut[class_label]
    ax.scatter(principalComponents[y==class_label][:,0],
               principalComponents[y==class_label][:,1],
               c=class_color, 
               s=15)
ax.legend(class_names)
ax.grid()

<font color=red> *Observations? Which classes would you expect to be difficult to classify correctly? How do you interpret the different region sizes occupied by different classes?*</font>

We observe that some classes are relatively well separated, others have a strong overlap. Interestingly, we also see that the space different classes occupy is very different for different classes. `Apoptosis` for instance seems to cover a large region in the feature space. Indeed, this is a highly variable class: dead cells can have a highly variable appearance.

#### t-distributed stochastic neighbor embedding

Next, we turn to a newer projection technique which is widely used in Machine Learning, but also in Computational Biology (in particular for sequencing data): t-SNE. Loosely speaking, one considers the probabilities $p_{ij}$ of two points $x_i$ and $x_j$ to be neighbors. t-SNE aims at finding a low-dimensional feature space, where the distribution of the corresponding $q_{ij}$ (the probability that $x^\prime_i$ and $x^\prime_j$ are neighbors in the low-dimensional feature representation) are as close as possible to $p_{ij}$ in the sense of KL-divergence:
\begin{equation}
KL(P\|Q) = \sum_{i\not= j}p_{ij}\log\frac{p_{ij}}{q_{ij}}
\end{equation}

In [ ]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(X)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('tSNE-component 1')
ax.set_ylabel('tSNE-component 2')
ax.set_title('tSNE')

class_names = name_to_label.keys()
for class_name in class_names:
    class_label = name_to_label[class_name]
    class_color = lut[class_label]
    ax.scatter(X_embedded[y==class_label][:,0],
               X_embedded[y==class_label][:,1],
               c=class_color, 
               s=15)
ax.legend(class_names)
ax.grid()

<font color=red> *How does this compare to the results obtained by PCA?*</font>

<font color=red> *Test different values of perplexity.*</font>

We observe that the grouping of the points corresponds much more to the actual class labels. We also observe that the distances between points are very different. While in PCA, the interphase cluster was very small, the apoptosis cluster occupied a large space. Here, the apoptosis cluster is much smaller, while the interphase cluster is becoming larger. We also observe that some classes seem to be difficult to separate by unsupervised techniques (they do not form individual clusters), in particular `MAP` that mixes happily with `Prometaphase` and `Apoptosis`. Furthermore, we see that `Anaphase` and `Metaphase` do not form natural clusters, but their distribution seems to be shifted (albeit overlapping). Such a scenario indicates that probably these classes would have been hard to find in an unsupervised setting but may be relatively easy to separate in a supervised setting.

#### Hierarchical clustering

As the heading indicates, this is not primarily a data visualization technique, but a method for unsupervised learning (i.e. a method to identify classes from data without annotation). Hierarchical clustering however has also nice visualization properties, because we can visualize the nested structures in our data and also display the feature combinations corresponding to the individual clusters in so scalled heatmaps. Thanks to its power for exploratory data analysis, hierarchical clustering is widely used in biology. 

In [ ]:
import seaborn as sns

In [ ]:
colorvec = [lut[label] for label in y]
cmap = sns.diverging_palette(10, 220, sep=80, n=30)

In [ ]:
g = sns.clustermap(X, method='ward', metric='euclidean', 
                   row_colors=colorvec, robust=True, cmap=cmap)

<font color=red> *Try different agglomeration functions: single, complete, average, centroid, Ward (check in the sklearn documentation, what they mean and conclude, which one you would like to use).* </font>

Again, we observe that some classes can be well distinguished, others are clustered together. When interpreting such a plot, it is essential to also look at the dendrogram. Actually, the true clustering information is represented by the dendrogram and not by the order of the rows. Indeed, we could flip the cluster order at any merging point of the dendrogram without changing the clustering result provided by the hierarchical clustering. 

In the above example we used Ward's method for agglomeration and euclidean distance (`method="ward"`, `metric="euclidean"`). In an unsupervised setting, we could certainly infer some of these groups from the data set without manual annotation. Others - as mentioned above - do not form individual clusters. In this case, it is essential to impose prior knowledge in form of manual annotations.

## Supervised Learning

We will now train classifiers on our training set. Here, we train Random Forest Classifiers (RF) and Support Vector Machines (SVM) in order to recognize nuclear morphologies.

#### Random Forests

Random Forests are ensemble methods, where decision trees are trained on random subsets of the training data, where each split is performed on one feature. This feature is chosen from a random subset of the features in order to optimize the purity of the resulting sets.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Now, we need to define the hyperparameters. In practice, on our kind of data, the most important parameter to be set is the number of trees in the forest. In general, the higher this number the better it is, but the resulting classifier will be slower also. Importantly, this parameter does not only affect training time, but also prediction time. For us however, this is not critical, and we set this parameter to 200. The second parameter is the `oob_score` which indicates whether to calculate the out-of-bag error. This is an error wich is only calculated on those samples that were left out for training the tree. Using the out-of-bag error allows us to avoid time-consuming cross-validation.

In [ ]:
rf = RandomForestClassifier(n_estimators=200, oob_score=True)

Now, we can train our classifier. 

In [ ]:
rf.fit(X, y)

In [ ]:
print('overall accuracy: %.2f' % rf.oob_score_)

We will now calculate the confusion matrix. 

In [ ]:
prediction_result = rf.oob_decision_function_

In [ ]:
classification_result = np.argmax(prediction_result, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

class_labels = np.array(sorted(label_to_name.keys()))
cm_rf = confusion_matrix(y, class_labels[classification_result])

print(', '.join([label_to_name[l] for l in class_labels]))
print(cm_rf)

#### Support Vector Machines (SVM)

Here, training is formulated as a convex optimization problem under constraints:

\begin{eqnarray*}
\min_{w,\xi} & & \|w\|^2 + C \sum_{i=1}^{N}\xi_i\\
\mbox{subject to} & & y_i(w^Tx_i + b) \geq 1 - \xi_i \quad i = 1, \ldots, N \\
& & \xi_i \geq 0 \quad i = 1, \ldots, N
\end{eqnarray*}

The only parameter to be fixed is $C$, which controls the compromise between classifcation errors and regularization (the higher $C$ is chosen, the more weight is given to correct classification of training samples). There is no way of guessing $C$ and so we need to test different values of $C$ and choose the value that leads to the best performance. 

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [ ]:
clf = svm.SVC(C=1, kernel='linear')

It is very important to evaluate the performance of the classifier by cross-validation: the training error is not equal to the test error. We are going to show this. First, we calculate the training error.

In [ ]:
clf.fit(X,y)

In [ ]:
print('accuracy measured on the training set: %.2f' % np.mean(clf.score(X,y)))

In [ ]:
score = cross_val_score(clf, X, y, cv=10)

In [ ]:
print('accuracy measured by 10-fold cross validation: %.2f' % np.mean(score))

In [ ]:
from sklearn.model_selection import cross_val_predict
res = cross_val_predict(clf, X, y)

cm_rf = confusion_matrix(y, res)

print(', '.join([label_to_name[l] for l in class_labels]))
print(cm_rf)

As expected, we see that the performance on the training set is better than on the test set. Indeed, evaluating the performance on the training set is a typical mistake in machine learning, as this always gives overoptimistic results. While in this case, this was easy to avoid, it is sometimes more challenging to know whether our performance evaluation is correct in that it does not intermingle training and test set.

Here, we obtain a very similar result to what we obtained from the RF classifier, even though we use a linear kernel and did not yet optimize the parameter $C$. This is not a typical situation however: in most cases, optimization of $C$ is crucial. 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'C':[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 100, 500]}

In [ ]:
gs = GridSearchCV(clf, parameters, cv=10)

In [ ]:
gs.fit(X, y)

In [ ]:
gs.best_score_

In [ ]:
gs.cv_results_['mean_test_score']

From here, we see that for this case, optimization of the parameter $C$ only marginally changes the accuracy. This is unusual, but it can happen. Nevertheless, it is essential to always check!

<font color=red> *Train a classifier on the principal components. Plot the accuracy depending on the number of the principal components used. For 4 components, test the effect of the parameter C.* </font>